# Text Analyzing 

When analyzing the text,there are ususally three major perspectives are considered. 
- first is the quantity measures, including the number of words or characters, the number or numbers, the freuency of the word and etc.  
- The other one the the text readability. text readability is an important factor to be considered, expecially for the quantative analysis study. The popular readability metrics include Flesch Kincaid Grade Level, Flesch Reading Ease, Gunning Fog Index, Dale Chall Readability, Automated Readability Index (ARI), Coleman Liau Index, Lisnear Write, and SMOG. Here's a package in python called **py-readability-metrics** which can enable us to get the score of all these mentioned metrics. 
- The third one is the content analysis of the text file, which LDA(laatent dirichlet allocaation method is a popular top  modeling used nowadays. 


## Installation 
~~~
pip install nltk
pip install py-readability-metrics
python -m nltk.downloader punkt
~~~

## Basic Usage 
~~~
from readability import Readability
import nltk
import string 
from nltk.corpus import stopwords  

with open('sample.txt', 'r') as f: 
    text = f.read() 
r = Readability(text) 
print( 
r.flesch_kincaid(),
r.flesch(),
r.gunning_fog(),
r.smog(),
r.coleman_liau(),
r.dale_chall(),
r.ari(),
r.linsear_write(),
r.smog(),sep = '\n')

# r.flesch().score
# r.flesch().ease 
# r.flesch().grade_levels
~~~ 
#### Reference:
https://py-readability-metrics.readthedocs.io/en/latest/flesch_kincaid.html 


In [ ]:
'''
Read all the txt files in the subfolders and mail folders. 
create a list of all the txt files 
'''
import pandas as pd
from readability import Readability
import nltk
import string 
from nltk.corpus import stopwords  
import os
from pathlib import Path
txt_list = []
for root, dirs, files in os.walk(r'main_directory'):
#     txt_list = [os.path.join(root,file) for file in files if file.endswith(".txt")]
    for file in files: 
        if file.endswith('.txt'): 
            txt_list.append(os.path.join(root,file))

In [ ]:
'''There are 10 BatchEngine files which are GB2312 encoding which cannot be parsed. 
And they need to be removed from the txt_list first before parsing all the other files''' 


import chardet  
for file in txt_list: 
    rawdata = open(file, 'rb').read()
    result = chardet.detect(rawdata)
    charenc = result['encoding'] 
    if charenc == None: 
        print(file, charenc, file = open(r"empty.txt","a")) 
    if charenc == 'GB2312': 
        print(file, charenc, file = open(r"GB2312.txt","a"))

with open('GB2312.txt') as f:
    lines = f.readlines()

batchengine_ls = [line.split(" ")[0] for line in lines]

## Remove batchengine ls from txt_list  
txt_list_clean = list(set(txt_list) - set(batchengine_ls))

In [ ]:
'''
Build up function to get all these 3 metrics for readability measures: 
1. flesch_kincaid
2. flesch_read_ease 
3. fog index 
'''
import glob 
ls_txt_name = [names.split("main directory of text files")[1] for names in txt_list_clean] 

def get_rd_metric(filename): 
    try: 
        with open(filename,'r') as f: 
            text = f.read()
            r = Readability(text)
        
            return r.flesch_kincaid().score,  r.flesch_kincaid().grade_level, \
                    r.flesch().score, r.flesch().grade_levels, \
                    r.gunning_fog().score, r.gunning_fog().grade_level 
    except Exception as e: 
        print(filename,e, file = open(r"error_log_0906.txt","a"))

In [ ]:
# store all the readability metrics in a list 

ls_all_metric = [get_rd_metric(filename) for filename in txt_list_clean] 

In [ ]:
# create a dataframe from the readability metrics list 

dic_metric = {"file_name": ls_txt_name,"metric":ls_all_metric}
df_metric = pd.DataFrame(dic_metric)


df_metric[['flesch_kincaid_score','flesch_kincaid_grade_level',
           'flesch_read_ease_score','flesch_read_ease_grade_level',
         'fog_score','fog_grade_level']] = pd.DataFrame(
    df_metric['metric'].tolist(), index = df_metric.index) 

## save pandas to csv showing chinese character, using utf_8_sig encoding 
df_metric.to_csv('metric_table.csv', encoding = 'utf_8_sig')

In [ ]:
## show full rows/full columns information

pd.set_option('display.max_colwidth',-1)
pd.set_option('display.max_rows',-1)

In [ ]:
df_metric.shape